## Problem Statement
UTKFace Dataset is a large-scale face dataset withalong age span (range from 0 to 116 years old). The objective is to classify each face based  on  gender  using  CNNs  on  Tensorflow  2.x  and  then  use OpenCV & Haar Cascade File to check the gender in real-time

## Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.models import Sequential

## Data Exploration:Analyze and explore the dataset

In [ ]:
path = "/kaggle/input/utkface-new/UTKFace/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [ ]:
print("Number of features",len(files[0].split("_"))-1)

## The shape of the dataset is(23708,3)
### where the number of images are 23708 and the features are 3 i.e., age, gender and race. Since we are doing gender detection we will only keep that feature

In [ ]:
images = []
ages = []
genders = []
for file in files:
    image = cv2.imread(path+'/'+file,1)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,dsize=(64,64))
    image = image.reshape((image.shape[0],image.shape[1],3))
    images.append(image)
    split_var = file.split('_')
    genders.append(int(split_var[1]) )

In [ ]:
# image = cv2.imread(path+'/'+files[0],1)
# image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.imshow(images[0])

In [ ]:
x_gender = list(set(genders))
y_gender = [genders.count(i) for i in x_gender]
plt.bar(x_gender,y_gender)
plt.show()

## Unique count of male and female

In [ ]:
print(y_gender)

## Visualize the images present in the dataset

In [ ]:
def display(img):
    plt.imshow(img)
idx = 2010
sample = images[idx]
print("Gender:",genders[idx])
display(sample)

## Prepare the dataset for the model

### Normalize the data 

In [ ]:
# pre processing
target = np.zeros((size),dtype='float32')
features = np.zeros((size,sample.shape[0],sample.shape[1],3),dtype = 'float32')
for i in range(size):
    target[i] = int(genders[i])
    features[i] = images[i]
features = features / 255
display(features[552])

In [ ]:
target[552]

## Encoding the labels: encoding is not required since they are in binary format

## Split the dataset using sklearn’s train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2,shuffle  = True)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

In [ ]:
y_train.shape

## Develop the model for recognizing the gender

In [ ]:
model=Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(64,64,3)))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss =['binary_crossentropy'],metrics=['accuracy'])

In [ ]:
h = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs = 30, batch_size=64,shuffle = True)

## Plot accuracy curves during training and validation

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Plot loss curves

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Save the model file

In [ ]:
model.save('model_weights.h5')

## Evaluate the model

In [ ]:
model.evaluate(x_test,y_test,verbose=0)

## Convert the model file to json format 

In [ ]:
json_config = model.to_json()

In [ ]:
new_model = model_from_json(json_config)

In [ ]:
##model.layers[0].weights

## Predict the Gender of the uploaded image

In [ ]:
def display(img):
    plt.imshow(img)
    
def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

def get_result(sample):
    sample = sample/255
    val = model.predict( np.array([ sample ]) )    
    gender = get_gender(val[0])
    print("Values:",val,"\nPredicted Gender:",gender)
    
    
indexes = [500]
for idx in indexes:
    sample = images[idx]
    display(sample)
    print("Actual Gender:",get_gender(genders[idx]))
    res = get_result(sample)

## Using Haar cascade to check in real time

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    
    if not ret:
        continue
    img= cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    
    #window for each face detected in the frame
    
    faces_detected = face_haar_cascade.detectMultiScale(img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        
        roi=img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        
        roi=cv2.resize(roi,(64,64))
        
        img_pixels = image.img_to_array(roi)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = get_gender(predictions[0])

        gender = ('male', 'female')
        predicted_gender = gender[max_index]

        cv2.putText(test_img, predicted_gender, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows